# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
import os

file_path = '../output_data/cities.csv'

# Check if the file exists
if not os.path.isfile(file_path):
    # If not, create the file or handle it according to your use case
    with open(file_path, 'w') as file:
        # You can write initial data to the file if needed
        file.write('Header1,Header2\n')

# Now, proceed with accessing the file
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,289.37,82,75,5.66,PT,1706733876
1,1,kodiak,57.7900,-152.4072,261.88,57,100,2.57,US,1706733876
2,2,lebu,-37.6167,-73.6500,291.48,76,70,5.54,CL,1706733877
3,3,port elizabeth,-33.9180,25.5701,294.44,87,0,3.58,ZA,1706733877
4,4,broome,42.2506,-75.8330,275.87,85,100,2.57,US,1706733878


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,289.37,82,75,5.66,PT,1706733876
1,1,kodiak,57.7900,-152.4072,261.88,57,100,2.57,US,1706733876
2,2,lebu,-37.6167,-73.6500,291.48,76,70,5.54,CL,1706733877
3,3,port elizabeth,-33.9180,25.5701,294.44,87,0,3.58,ZA,1706733877
4,4,broome,42.2506,-75.8330,275.87,85,100,2.57,US,1706733878


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display


# Configure the map plot
humidity_map = city_data_df.hvplot.scatter(
    x='Lng', y='Lat', c='Humidity', cmap='viridis',
    size='Humidity', alpha=0.7,
    title='City Humidity Map', height=500, width=700,
    colorbar=True, xlabel='Longitude', ylabel='Latitude'
)



# Display the map
humidity_map

:Scatter   [Lng]   (Lat,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] > 35) & (city_data_df['Humidity'] > 50)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,289.37,82,75,5.66,PT,1706733876
1,1,kodiak,57.7900,-152.4072,261.88,57,100,2.57,US,1706733876
2,2,lebu,-37.6167,-73.6500,291.48,76,70,5.54,CL,1706733877
3,3,port elizabeth,-33.9180,25.5701,294.44,87,0,3.58,ZA,1706733877
4,4,broome,42.2506,-75.8330,275.87,85,100,2.57,US,1706733878


In [6]:
print(city_data_df[['Max Temp', 'Humidity']].describe())


         Max Temp    Humidity
count  565.000000  565.000000
mean   285.038248   73.330973
std     16.937469   20.428178
min    236.670000   12.000000
25%    275.230000   63.000000
50%    290.550000   78.000000
75%    298.570000   88.000000
max    310.140000  100.000000


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ribeira grande,PT,38.5167,-28.7000,82,
1,kodiak,US,57.7900,-152.4072,57,
2,lebu,CL,-37.6167,-73.6500,76,
3,port elizabeth,ZA,-33.9180,25.5701,87,
4,broome,US,42.2506,-75.8330,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 5000  
params = {
    "type": "accommodation",
    "limit": 1,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["bias"] = f"circle:{radius}@{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribeira grande - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
tofol - nearest hotel: No hotel found
kununurra - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
chipiona - nearest hotel: No hotel found
college - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
lobujya - nearest hotel: No hotel found
kapuskasing - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
tolanaro - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
nuku'alofa - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
dompu - nearest hotel: No hotel found
nizhnekamsk - nearest hotel: No hotel found
mersa matruh - nearest hotel: No h

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ribeira grande,PT,38.5167,-28.7000,82,No hotel found
1,kodiak,US,57.7900,-152.4072,57,No hotel found
2,lebu,CL,-37.6167,-73.6500,76,No hotel found
3,port elizabeth,ZA,-33.9180,25.5701,87,No hotel found
4,broome,US,42.2506,-75.8330,85,No hotel found
...,...,...,...,...,...,...
560,yatou,CN,37.1500,122.3833,55,No hotel found
561,ambon,ID,-3.6954,128.1814,14,No hotel found
562,along,IN,28.1667,94.7667,95,No hotel found
563,florence,IT,43.7667,11.2500,84,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE